In [5]:
import xml.etree.ElementTree as ET

mjMINVAL = 1e-6  # Example minimum value, adjust as needed

def validate_mjcf(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    for body in root.findall('.//body'):
        mass = float(body.get('mass', '0'))
        if mass < mjMINVAL:
            raise ValueError(f"Mass for body {body.get('name')} is below mjMINVAL.")
        
        inertial = body.find('inertial')
        if inertial is not None:
            inertia = [float(val) for val in inertial.get('diaginertia', '0 0 0').split()]
            if any(val < mjMINVAL for val in inertia):
                raise ValueError(f"Inertia for body {body.get('name')} is below mjMINVAL.")

# Replace 'your_mjcf_path.xml' with the path to your MJCF file
validate_mjcf(r'D:\Mujoco\fruitfly\flybody\flybody\fruitfly\assets\fruitfly.xml')


ValueError: Mass for body thorax is below mjMINVAL.

In [15]:
import xml.etree.ElementTree as ET

mjMINVAL = 1e-6  # Example minimum value, adjust as needed

def validate_mjcf(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    for geom in root.findall('.//geom'):
        geom_name = geom.get('name')
        if geom_name is not None:
            mass = float(geom.get('mass', '0'))
            if mass < mjMINVAL:
                raise ValueError(f"Mass for geom {geom_name} is below mjMINVAL.")
        else:
            print("Warning: Found a geom element without a name attribute.")

# Replace 'your_mjcf_path.xml' with the path to your MJCF file
validate_mjcf('CyberMiceJointActuated_2.xml')

ValueError: Mass for geom SpineRibs is below mjMINVAL.

In [16]:
import os
import xml.etree.ElementTree as ET

mjMINVAL = 1e-6  # Example minimum value, adjust as needed


def validate_mjcf(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    mistakes = []
    
    for geom in root.findall('.//geom'):
        geom_name = geom.get('name')
        if geom_name is not None:
            mass = float(geom.get('mass', '0'))
            if mass < mjMINVAL:
                mistakes.append(f"Mass for geom {geom.get('name')} in file {file_path} is below mjMINVAL.")
                # raise ValueError(f"Mass for geom {geom_name} is below mjMINVAL.")
        else:
            print("Warning: Found a geom element without a name attribute.")

    return mistakes


# Usage
all_mistakes = validate_mjcf(r'CyberMiceJointActuated_2.xml')
for mistake in all_mistakes:
    print(mistake)

Mass for geom SpineRibs in file D:\Mujoco\CyberMice\assets\CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom RScapula in file D:\Mujoco\CyberMice\assets\CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom RHumerus in file D:\Mujoco\CyberMice\assets\CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom RUlna in file D:\Mujoco\CyberMice\assets\CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom RRadius in file D:\Mujoco\CyberMice\assets\CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom RCarpi in file D:\Mujoco\CyberMice\assets\CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom RFinger in file D:\Mujoco\CyberMice\assets\CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom RClavicle in file D:\Mujoco\CyberMice\assets\CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom LScapula in file D:\Mujoco\CyberMice\assets\CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom LHumerus in file D:\Mujoco\CyberMice\

In [3]:
import xml.etree.ElementTree as ET
import math

mjMINVAL = 1e-6  # Example minimum value, adjust as needed

def calculate_mass_from_density(geom):
    density = float(geom.get('density', '0'))
    if density == 0:
        return 0
    
    geom_type = geom.get('type')
    if geom_type == 'capsule':
        radius = float(geom.get('size').split()[0])
        height = float(geom.get('size').split()[1]) * 2  # Assuming the second size parameter is half-height
        volume = math.pi * radius**2 * (2 * radius + height)
    else:
        # Add calculations for other geom types if necessary
        volume = 0  # Placeholder for unsupported geom types
    
    return density * volume

def validate_mjcf(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    mistakes = []
    
    for geom in root.findall('.//geom'):
        geom_name = geom.get('name', 'Unnamed geom')
        mass = geom.get('mass')
        if mass is None:
            mass = calculate_mass_from_density(geom)
        else:
            mass = float(mass)
        
        if mass < mjMINVAL:
            mistakes.append(f"Mass for geom {geom_name} in file {file_path} is below mjMINVAL.")
    
    for body in root.findall('.//body'):
        inertial = body.find('inertial')
        if inertial is not None:
            mass = float(inertial.get('mass', '0'))
            if mass < mjMINVAL:
                mistakes.append(f"Mass for body {body.get('name', 'Unnamed body')} in file {file_path} is below mjMINVAL.")
            
            for attr in ['ixx', 'iyy', 'izz']:
                inertia = float(inertial.get(attr, '0'))
                if inertia < mjMINVAL:
                    mistakes.append(f"Inertia {attr} for body {body.get('name', 'Unnamed body')} in file {file_path} is below mjMINVAL.")
    
    return mistakes

# Replace 'your_mjcf_path.xml' with the path to your MJCF file
mistakes = validate_mjcf(r'CyberMiceJointActuated_2.xml')
for mistake in mistakes:
    print(mistake)

if not mistakes:
    print("No mistakes found.")


Mass for geom Unnamed geom in file CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom RScapula in file CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom RHumerus in file CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom RUlna in file CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom RRadius in file CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom RCarpi in file CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom RFinger in file CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom RClavicle in file CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom LScapula in file CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom LHumerus in file CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom LUlna in file CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom LRadius in file CyberMiceJointActuated_2.xml is below mjMINVAL.
Mass for geom LCarpi in file CyberMiceJointActuated_2.xml 